# Importing modules

In [2]:
from ipywidgets import widgets
from IPython.display import display

In [3]:
from plotly.offline import download_plotlyjs, init_notebook_mode, iplot
import plotly.plotly as py
import plotly.graph_objs as go
import plotly.widgets
init_notebook_mode(connected=True)
plot_config={'showLink':False}

<IPython.core.display.Javascript object>

In [4]:
import pandas as pd

In [5]:
from elasticsearch import Elasticsearch
from elasticsearch import client
es = Elasticsearch('#') #Elasticsearch server address purged for publication

# Defining the helper functions

In [8]:
# Flattens any arbitrarily nested dictionary
def flatify(json):
    
    def engine(data, header=''):
        if type(data) is list:
            c=0
            for obj in data:
                engine(obj, '{}[{}]'.format(header,str(c)))
                c+=1
                
        elif type(data) is dict:
            for k,v in data.items():
                engine(v, header+k+'.')
        
        else:
            row[header]=data
    row={}            
    engine(json)
    return row

In [10]:
# Builds an alternative association of key:value if an alternative key is present
def build_col(key, data, altkey=None):
    if altkey != None:
        if not key in data:
            key = altkey
            
    def engine(key, data):
        if not key in data:
            data[key]=''
        return data[key]
    
    output = engine(key, data)
    return output

In [11]:
# Builds the row for each row based on the flattened dictionary using "flatify" function
def build_row(row_data):
    atc = build_col('therapeutic_class.[0]tc_atc_number.', row_data)
    din = build_col('drug_identification_number.', row_data)
    history_date = build_col('status_detail.[0]history_date.', row_data)
    original_date = build_col('status_detail.[0]original_market_date.', row_data, 'status_marketed_date.[0]')
    status = build_col('status_detail.[0]status.',row_data)
    return [atc,din,history_date,original_date,status]

# Sourcing and Wrangling the data

## Querying elasticsearch for the data

In [6]:
indexname = 'dpd_drug'
q = {
  "query": {
    "query_string": {
      "query": "(status_current:Marketed OR status_current:\"Cancelled Post Market\" OR status_current:Dormant) AND class:Human"
    }
  }
}
client.IndicesClient.put_settings(es, body={"max_result_window":50000}, index="dpd_drug")
raw_data = es.search(body=q, index=indexname, size=50000)
client.IndicesClient.put_settings(es, body={"max_result_window":10000}, index="dpd_drug")

query_size = raw_data['hits']['total']
hits = raw_data['hits']['hits']
print(query_size)

38254


## Dataframe build

Building an array of rows to append to the dataframe:

In [ ]:
#Building an array of rows to append to the dataframe
# c=0
rows=[]
for hit in hits:
    row = flatify(hit['_source'])
#     print(c+1)
    try:
        rows.append(build_row(row))
        c+=1
    except Exception as e:
        print('{}:{} | hit# {}'.format(type(e),e.args, c))
    

Defining the columns of dataframe 'df':

In [13]:
# Defining the columns of dataframe 'df'
df = pd.DataFrame(columns=['atc','din','history_date','original_market_date','status'], data = rows)
df.loc[:,'original_market_date'] = pd.to_datetime(df.loc[:,'original_market_date'], yearfirst=True)
df.loc[:,'history_date'] = pd.to_datetime(df.loc[:,'history_date'], yearfirst=True)

Dropping rows where the original market date has a null value.  This only amounted to a handful of DINs which have the current status of being Cancelled Post-Market

In [15]:
# Dropping rows where the original market date has a null value
# This only amounted to a handful of DINs which have the current status of being Cancelled Post-Market
df.dropna(subset=['original_market_date'], inplace=True)

Since the queried data are assumed to have been marketed (Marketed, Cancelled Post-Market, Dormant), every DINs receives an "entered_counter" value of 1.  If the current status is not "Marketed", its status is assumed to be not available/taken off the market and is assigned an "exit_counter" value of -1.  Otherwise, it is assumed to be still on the market and an "exit_counter" value is set to 0.

In [16]:
# Add counter stacks to calculate market entry/exit
df['entered_counter'] = 1
df.loc[df.status!='Marketed','exit_counter'] = -1
df.loc[df.status=='Marketed','exit_counter'] = 0

In [18]:
df.head()

,atc,din,history_date,original_market_date,status,entered_counter,exit_counter
0,N02BA01,00010332,2018-08-03,1951-12-31,Dormant,1,-1.0
1,G03CA57,00265470,2017-07-27,1963-12-31,Dormant,1,-1.0
2,S01XA20,02026538,2017-07-24,1994-12-31,Dormant,1,-1.0
3,N01BB02,01947060,2017-04-27,1994-12-31,Dormant,1,-1.0
4,N02BE01,01904140,2017-07-25,1995-12-31,Dormant,1,-1.0


In [17]:
# Function to sorts the dataframe by a defined column, then reindexing the dataframe as the column.
# This is a helper function to build the final dataframe used for plotting
def preset_df(df,column):
    df = df.sort_values(by=column)
    df = df.set_index(column)
    return df

Reduce the ATC grouping by the first level by treating the column values as string and slicing after the first letter.

In [20]:
# Reduce the ATC grouping by the first level by treating the column values as string and slicing after the first letter.
df['atc']=df['atc'].astype(str).str[:1]

Function to build a new dataframe for each ATC code filtered from the dataframe 'df':

In [27]:
# Function to build a new dataframe for each ATC code filtered from the dataframe 'df'
def df_by_atc(df):
    df1 = df.copy()
    df2 = df1.copy()
    df1 = preset_df(df1, 'original_market_date')
    df2 = preset_df(df2, 'history_date')
    df1['entered_sum']=df1.entered_counter.resample('D').sum()
    df1.reset_index(level=0, inplace=True)
    df1['date']=df1['original_market_date']
    df2['exit_sum']=df2.exit_counter.resample('D').sum()
    df2.reset_index(level=0, inplace=True)
    df2['date']=df2['history_date']
    df1.drop_duplicates('original_market_date', inplace=True)
    df1_pr = pd.DataFrame(data={"date":df1['date'],"counter":df1['entered_sum']})
    # df1_pr = preset_df(df1_pr, 'date')
    # df1_pr.reset_index(level=0,inplace=True)
    df2.drop_duplicates('history_date', inplace=True)
    df2_pr = pd.DataFrame(data={"date":df2['date'],"counter":df2['exit_sum']})
    # df2_pr = preset_df(df2_pr, 'date')
    plot_df = pd.concat([df1_pr,df2_pr])
    # plot_df.sort_values(by='date', inplace=True)
    plot_df=preset_df(plot_df, 'date')
    plot_df['market_count'] = plot_df.counter.resample('D').sum()
    plot_df.reset_index(level=0, inplace=True)
    plot_df.drop_duplicates('date', inplace=True)
    plot_df=preset_df(plot_df,'date')
    plot_df['market_count_cum']=plot_df['market_count'].cumsum()
    return plot_df

In [33]:
# Visualizing the ATC codes in order to build the dataframes for each
sorted(df.atc.unique().tolist())

['', 'A', 'B', 'C', 'D', 'G', 'H', 'J', 'L', 'M', 'N', 'P', 'R', 'S', 'V']

In [38]:
# Passing each filtered dataframe into the function for plotting
df_  = df_by_atc(df.loc[df['atc'] == ''])
df_A = df_by_atc(df.loc[df['atc'] == 'A'])
df_B = df_by_atc(df.loc[df['atc'] == 'B'])
df_C = df_by_atc(df.loc[df['atc'] == 'C'])
df_D = df_by_atc(df.loc[df['atc'] == 'D'])
df_G = df_by_atc(df.loc[df['atc'] == 'G'])
df_H = df_by_atc(df.loc[df['atc'] == 'H'])
df_J = df_by_atc(df.loc[df['atc'] == 'J'])
df_L = df_by_atc(df.loc[df['atc'] == 'L'])
df_M = df_by_atc(df.loc[df['atc'] == 'M'])
df_N = df_by_atc(df.loc[df['atc'] == 'N'])
df_P = df_by_atc(df.loc[df['atc'] == 'P'])
df_R = df_by_atc(df.loc[df['atc'] == 'R'])
df_S = df_by_atc(df.loc[df['atc'] == 'S'])
df_V = df_by_atc(df.loc[df['atc'] == 'V'])

# Plotting the graph

## Building the variables for each plots

In [68]:
trace_ = go.Scatter(
    x=df_.index,
    y=df_['market_count_cum'],
    name="No code"
)
trace_A = go.Scatter(
    x=df_A.index,
    y=df_A['market_count_cum'],
    name="A"
)
trace_B = go.Scatter(
    x=df_B.index,
    y=df_B['market_count_cum'],
    name="B"
)
trace_C = go.Scatter(
    x=df_C.index,
    y=df_C['market_count_cum'],
    name="C"
)
trace_D = go.Scatter(
    x=df_D.index,
    y=df_D['market_count_cum'],
    name="D"
)
trace_G = go.Scatter(
    x=df_G.index,
    y=df_G['market_count_cum'],
    name="G"
)
trace_H = go.Scatter(
    x=df_H.index,
    y=df_H['market_count_cum'],
    name="H"
)
trace_J = go.Scatter(
    x=df_J.index,
    y=df_J['market_count_cum'],
    name="J"
)
trace_L = go.Scatter(
    x=df_L.index,
    y=df_L['market_count_cum'],
    name="L"
)
trace_M = go.Scatter(
    x=df_M.index,
    y=df_M['market_count_cum'],
    name="M"
)
trace_N = go.Scatter(
    x=df_N.index,
    y=df_N['market_count_cum'],
    name="N"
)
trace_P = go.Scatter(
    x=df_P.index,
    y=df_P['market_count_cum'],
    name="P"
)
trace_R = go.Scatter(
    x=df_R.index,
    y=df_R['market_count_cum'],
    name="R"
)
trace_S = go.Scatter(
    x=df_S.index,
    y=df_S['market_count_cum'],
    name="S"
)
trace_V = go.Scatter(
    x=df_V.index,
    y=df_V['market_count_cum'],
    name="V"
)

layout = go.Layout(
    legend=dict(
        x=0,
        y=1,
        bgcolor='rgba(239,239,239,0.7)',
        bordercolor='#FFFFFF',
        borderwidth=2
    ),
    showlegend=True,
    title="Market availability by ATC group",
    xaxis={'title':'Time'},
    yaxis={'title':'Total unique DINs'},
)

trace_data = [trace_,
              trace_A,
              trace_B,
              trace_C,
              trace_D,
              trace_G,
              trace_H,
              trace_J,
              trace_L,
              trace_M,
              trace_N,
              trace_P,
              trace_R,
              trace_S,
              trace_V,]

fig = go.FigureWidget(data=trace_data,layout=layout)

In [82]:
# set to None if getting warnings
# pd.options.mode.chained_assignment = None

## Visuals/Final Plot

In [69]:
widgets.VBox([fig])

    'data': [{'name': 'No code',
              'type': 'scatter',
              …